In [1]:
max_item_count = 24
rarity_thershold = 135

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import defaultdict

In [3]:
csv_file_path_train = './train_dataset.csv'
csv_file_path_test = './submission_data_x.csv'

df_train = pd.read_csv(csv_file_path_train)
df_test = pd.read_csv(csv_file_path_test)

C:\Users\ethem\AppData\Local\Temp\ipykernel_16776\896890968.py:4: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,20,21,22,23,24,59,60,61,62,63,64,65,66,67,68,69,70,71,72,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,102,103,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(csv_file_path_train)
C:\Users\ethem\AppData\Local\Temp\ipykernel_16776\896890968.py:5: DtypeWarning: Columns (97,105,107,108,109,110,111,112,113,114,116,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(csv_file_path_test)


In [4]:
df_train = pd.read_csv(csv_file_path_train)

C:\Users\ethem\AppData\Local\Temp\ipykernel_16776\222863778.py:1: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,20,21,22,23,24,59,60,61,62,63,64,65,66,67,68,69,70,71,72,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,102,103,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(csv_file_path_train)


In [5]:
# Manage the TEST data
col_list = ["item", "make", "model", "goods_code"]
all_colmns_to_append = []
for clmn_name in col_list:

    # Extract uniqueness and repetiteveness
    row_count = df_test.shape[0]
    count_dict = defaultdict(int)
    
    cols = {clmn_name+str(i) for i in range(1,max_item_count+1)}

    for col in cols:
        column_values = df_test[col]
        # del df_test[col]
        for row in range(row_count - 1):
            item = column_values[row]
            if pd.notna(item):
                count_dict[str(item)] += 1

    count_dict = sorted(count_dict.items(), key=lambda x: x[1], reverse=True)
    
    # ADD COLUMNS  
    for key, count in count_dict:
        if count > rarity_thershold:
            all_colmns_to_append.append(str(key))

all_colmns_to_append = list(set(all_colmns_to_append))
new_data = pd.DataFrame(0, index=df_test.index, columns=all_colmns_to_append)
df_test = pd.concat([df_test, new_data], axis=1)

for clmn_name in col_list:
    cols = {clmn_name+str(i) for i in range(1,max_item_count+1)}
    # FILL NEWLY ADDED COLUMNS
    for index, row in df_test.iterrows():
        for col in cols:
            cell = row[col]
            if not pd.notna(cell):
                break
            if cell in df_test.columns:
                df_test.at[index, cell] = 1

    # DROP OLD COLUMNS
    for col in cols:
        del df_test[col]

In [6]:
# Manage the TRAIN data

# ADD COLUMNS
cols_df_test = list(df_test.columns)
cols_df_train = list(df_train.columns)
cols_add_train = []

for col in cols_df_test:
    if col not in cols_df_train:
        cols_add_train.append(col)

new_data = pd.DataFrame(0, index=df_train.index, columns=cols_add_train)
df_train = pd.concat([df_train, new_data], axis=1)

for clmn_name in col_list:

    cols = {clmn_name+str(i) for i in range(1,max_item_count+1)}

    # # FILL NEWLY ADDED COLUMNS
    for index, row in df_train.iterrows():
        for col in cols:
            cell = row[col]
            if not pd.notna(cell):
                break
            if cell in df_train.columns:
                df_train.at[index, cell] = 1

    # DROP OLD COLUMNS
    for col in cols:
        del df_train[col]

In [7]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import average_precision_score

df_train = df_train.sample(frac=1).reset_index(drop=True)

# train_x ve train_y'yi yeniden tanımla (karıştırılmış df_train'den sonra)
train_y = df_train['fraud_flag']
train_x = df_train.drop(columns=['fraud_flag'])

# %20'sini yeni bir test seti olarak ayır
X_train, X_new_test, y_train, y_new_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [13]:
# RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_new_test_probabilities = rf_model.predict_proba(X_new_test)[:, 1]

# XGBoost modeli
xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)
xgb_new_test_probabilities = xgb_model.predict_proba(X_new_test)[:, 1]


results_df = pd.DataFrame({'Actual': y_new_test, 
                           'RF_Predicted_Probability': rf_new_test_probabilities,
                           'XGB_Predicted_Probability': xgb_new_test_probabilities})

# Sonuçları bir dosyaya yaz
results_df.to_csv('new_test_results.txt', index=False, sep='\t')

# RandomForestClassifier için average precision score hesapla
rf_average_precision = average_precision_score(results_df['Actual'], results_df['RF_Predicted_Probability'])
print(f"Random Forest Average Precision Score: {rf_average_precision}")

# XGBoost için average precision score hesapla
xgb_average_precision = average_precision_score(results_df['Actual'], results_df['XGB_Predicted_Probability'])
print(f"XGBoost Average Precision Score: {xgb_average_precision}")


Random Forest Average Precision Score: 0.18594022900214496
XGBoost Average Precision Score: 0.2148908889745039


In [14]:
from sklearn.ensemble import VotingClassifier

# Voting Classifier oluştur
voting_clf = VotingClassifier(estimators=[
    ('random_forest', rf_model),
    ('xgboost', xgb_model)
], voting='soft')  # 'soft' oylama, sınıf olasılıklarının ortalamasını alır

# Voting Classifier'ı eğit
voting_clf.fit(X_train, y_train)

# Voting Classifier'ın tahminlerini al
voting_new_test_probabilities = voting_clf.predict_proba(X_new_test)[:, 1]

# Voting Classifier için average precision score hesapla
voting_average_precision = average_precision_score(y_new_test, voting_new_test_probabilities)
print(f"Voting Classifier Average Precision Score: {voting_average_precision}")


Voting Classifier Average Precision Score: 0.2355289369051183


In [ ]:
xgb_new_test_probabilities=voting_clf.predict_proba(df_test)[:, 1]

In [ ]:
# Tahmin olasılıklarını prediction.txt dosyasına yaz
with open("sample_submission.csv", "r") as file:
    lines = file.readlines()

# Olasılık değerlerini içeren bir liste olduğunu varsayalım
# Örneğin:
# fraud_proba = [0.08, 0.0, 0.003333333333333333, 0.00030535604963342346, 0.0, ...]

# Dosyanın sonuna olasılık değerlerini ekleyerek yeni bir liste oluştur
new_lines = []
for i, prob in enumerate(xgb_new_test_probabilities):
    new_lines.append(f"{lines[i + 1].strip()}{prob}\n")  # İlk satır başlık olduğu için 1'den başlayalım

# Yeni satırları dosyaya yaz
with open("voting.csv", "w") as file:
    file.write("ID,fraud_flag\n")  # Başlık satırını yaz
    file.writelines(new_lines)
